In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.history_aware_retriever import create_history_aware_retriever
import pickle
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# from utils.prompts import  contextualize_q_prompt, qa_prompt
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [ ]:
def initiate():
    with open('Data/db.pkl','rb') as f: pkl = pickle.load(f)
    
    vectorstore = FAISS.deserialize_from_bytes(serialized=pkl, embeddings=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
    llm=ChatOpenAI(model="gpt-3.5-turbo")

    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    store = {}


    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]


    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    
    return conversational_rag_chain



In [ ]:
def response(conversational_rag_chain,user_query,ssid):
    return conversational_rag_chain.invoke(
            {"input": user_query},
            config={
                "configurable": {"session_id":f"{ssid}"}
            },
        )['answer']


In [ ]:
print("Welcome to the chatbot. Type 'exit' to quit.")
print("Type 'clear' to clear the conversation history.")
chain = initiate()
ssid = 1
while True:
    user_query = input("Enter your query: ")
    if user_query == 'exit': break
    if user_query == 'clear':
        print("Conversation history cleared. Type 'exit' to quit.")
        ssid += 1
    print(response(chain,user_query,ssid))

In [41]:
sql_system_prompt = (
    """
    You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:

{table_info}

/*
3 rows from cb_products table:
ID	code	name	description	url	image_url	category_names	category_label	tag_names
9237	B4705N	SHOWER HEAD TRONICO 250	• Contemporary organic • Ø 250 mm – L-M 415 • Intelligent water distribution resulting in minimal wa	https://www.nikles.com/product/shower-head-tronico-250	https://www.nikles.com/wp-content/uploads/2020/05/B4705N1-scaled.jpg	Head Showers	Tronico	Airdrop, Easy-to-clean
9251	BLS.001.07N	LIPS CASCADE SHOWER HEAD - MATTE BLACK	• Outstanding contemporary • Dimensions: 215 x 82.5 • Cascade spray with intelligent water • Materia	https://www.nikles.com/product/lips-cascade-shower-head-black	https://www.nikles.com/wp-content/uploads/2022/03/BLS.001.07N.jpg	Head Showers	Lips	Cascade Shower
	LIPS CASCADE SHOWER HEAD - RED	• Outstanding contemporary • Dimensions: 215 x 82.5 • Cascade spray with intelligent water • Materia	https://www.nikles.com/product/lips-cascade-shower-head-red	https://www.nikles.com/wp-content/uploads/2022/03/BLS.001.RRN_.jpg	Head Showers	Lips	Cascade Shower
*/

Question: {input}
"""
)
sql_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sql_system_prompt),
        # MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# sql_prompt.partial(top_k=3)

ChatPromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': 3}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'table_info', 'top_k'], template='\n    You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which col

In [66]:
from langchain_community.utilities import SQLDatabase
mysql_uri = 'mysql+mysqlconnector://root:@127.0.0.1:3306/products'
db = SQLDatabase.from_uri(mysql_uri)

In [67]:
def get_schema(_):
    schema = db.get_context()['table_info']
    return schema


In [62]:
context = db.get_context()
print(list(context))
print(context["table_info"])

['table_info', 'table_names']

CREATE TABLE cb_products (
	`ID` BIGINT(20) UNSIGNED NOT NULL, 
	code VARCHAR(255) NOT NULL, 
	name TEXT, 
	description TEXT, 
	url VARCHAR(255), 
	image_url TEXT, 
	category_names TEXT, 
	category_label TEXT, 
	tag_names TEXT, 
	PRIMARY KEY (`ID`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_general_ci

/*
3 rows from cb_products table:
ID	code	name	description	url	image_url	category_names	category_label	tag_names
9237	B4705N	SHOWER HEAD TRONICO 250	• Contemporary organic • Ø 250 mm – L-M 415 • Intelligent water distribution resulting in minimal wa	https://www.nikles.com/product/shower-head-tronico-250	https://www.nikles.com/wp-content/uploads/2020/05/B4705N1-scaled.jpg	Head Showers	Tronico	Airdrop, Easy-to-clean
9251	BLS.001.07N	LIPS CASCADE SHOWER HEAD - MATTE BLACK	• Outstanding contemporary • Dimensions: 215 x 82.5 • Cascade spray with intelligent water • Materia	https://www.nikles.com/product/lips-cascade-shower-head-black	https://www.nikl

In [8]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [43]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db,sql_prompt)
# chain.get_prompts()[0].pretty_print()

In [61]:
table_info

NameError: name 'table_info' is not defined

In [68]:
temp_prompt = ChatPromptTemplate.from_template(sql_system_prompt)
sql_chain = (
    RunnablePassthrough()
    | temp_prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)


In [69]:
sql_chain.invoke({"question": "tell me about shower heads"})

KeyError: "Input to ChatPromptTemplate is missing variables {'input', 'top_k', 'table_info'}.  Expected: ['input', 'table_info', 'top_k'] Received: ['question']"

['table_info', 'table_names']

CREATE TABLE cb_products (
	`ID` BIGINT(20) UNSIGNED NOT NULL, 
	code VARCHAR(255) NOT NULL, 
	name TEXT, 
	description TEXT, 
	url VARCHAR(255), 
	image_url TEXT, 
	category_names TEXT, 
	category_label TEXT, 
	tag_names TEXT, 
	PRIMARY KEY (`ID`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_general_ci

/*
3 rows from cb_products table:
ID	code	name	description	url	image_url	category_names	category_label	tag_names
9237	B4705N	SHOWER HEAD TRONICO 250	• Contemporary organic • Ø 250 mm – L-M 415 • Intelligent water distribution resulting in minimal wa	https://www.nikles.com/product/shower-head-tronico-250	https://www.nikles.com/wp-content/uploads/2020/05/B4705N1-scaled.jpg	Head Showers	Tronico	Airdrop, Easy-to-clean
9251	BLS.001.07N	LIPS CASCADE SHOWER HEAD - MATTE BLACK	• Outstanding contemporary • Dimensions: 215 x 82.5 • Cascade spray with intelligent water • Materia	https://www.nikles.com/product/lips-cascade-shower-head-black	https://www.nikl

In [ ]:
prompt_with_context = chain.get_prompts()[0].partial(table_info=context["table_info"])
print(prompt_with_context.pretty_repr())

In [20]:
chain = create_sql_query_chain(llm, db, prompt_with_context)
chain.invoke({"question": "how many shower heads are there?"})

ValueError: Prompt must have input variables: 'input', 'top_k', 'table_info'. Received prompt with input variables: ['input']. Full prompt:

input_variables=['input'] partial_variables={'top_k': '5', 'table_info': '\nCREATE TABLE cb_products (\n\t`ID` BIGINT(20) UNSIGNED NOT NULL, \n\tcode VARCHAR(255) NOT NULL, \n\tname TEXT, \n\tdescription TEXT, \n\turl VARCHAR(255), \n\timage_url TEXT, \n\tcategory_names TEXT, \n\tcategory_label TEXT, \n\ttag_names TEXT, \n\tPRIMARY KEY (`ID`)\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_general_ci\n\n/*\n3 rows from cb_products table:\nID\tcode\tname\tdescription\turl\timage_url\tcategory_names\tcategory_label\ttag_names\n9237\tB4705N\tSHOWER HEAD TRONICO 250\t• Contemporary organic • Ø 250 mm – L-M 415 • Intelligent water distribution resulting in minimal wa\thttps://www.nikles.com/product/shower-head-tronico-250\thttps://www.nikles.com/wp-content/uploads/2020/05/B4705N1-scaled.jpg\tHead Showers\tTronico\tAirdrop, Easy-to-clean\n9251\t

In [13]:
examples = [
    {"input": "List all products.", "query": "SELECT * FROM cb_products;"},
    {
        "input": "Find all shower kits",
        "query": "SELECT * FROM cb_products WHERE ID = (SELECT ID FROM cb_products WHERE category_names = 'Shower Kits');",
    },
]

In [18]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    # prefix="You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than {top_k} rows.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries.",
    prefix=chain.get_prompts()[0].partial(table_info=context["table_info"]),
    suffix="User input: {input}\nSQL query: ",
    input_variables=["input", "top_k", "table_info"],
)

KeyError: 'prefix'

In [15]:
print(prompt.format(input="how many artists are there?", top_k=3, table_info="foo"))

You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than 3 rows.

Here is the relevant table info: foo

Below are a number of examples of questions and their corresponding SQL queries.

User input: List all products.
SQL query: SELECT * FROM cb_products;

User input: Find all shower kits
SQL query: SELECT * FROM cb_products WHERE ID = (SELECT ID FROM cb_products WHERE category_names = 'Shower Kits');

User input: how many artists are there?
SQL query: 


In [19]:
chain = create_sql_query_chain(llm, db, prompt_with_context)
chain.invoke({"question": "how many shower heads are there?"})

ValueError: Prompt must have input variables: 'input', 'top_k', 'table_info'. Received prompt with input variables: ['input']. Full prompt:

input_variables=['input'] partial_variables={'top_k': '5', 'table_info': '\nCREATE TABLE cb_products (\n\t`ID` BIGINT(20) UNSIGNED NOT NULL, \n\tcode VARCHAR(255) NOT NULL, \n\tname TEXT, \n\tdescription TEXT, \n\turl VARCHAR(255), \n\timage_url TEXT, \n\tcategory_names TEXT, \n\tcategory_label TEXT, \n\ttag_names TEXT, \n\tPRIMARY KEY (`ID`)\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_general_ci\n\n/*\n3 rows from cb_products table:\nID\tcode\tname\tdescription\turl\timage_url\tcategory_names\tcategory_label\ttag_names\n9237\tB4705N\tSHOWER HEAD TRONICO 250\t• Contemporary organic • Ø 250 mm – L-M 415 • Intelligent water distribution resulting in minimal wa\thttps://www.nikles.com/product/shower-head-tronico-250\thttps://www.nikles.com/wp-content/uploads/2020/05/B4705N1-scaled.jpg\tHead Showers\tTronico\tAirdrop, Easy-to-clean\n9251\t